In [13]:
import bw2io as bi
import bw2data as bd
import bw2calc as bc
from bw_temporalis import easy_timedelta_distribution, TemporalisLCA

In [14]:
bd.projects.set_current('tictac')

In [20]:
bd.databases

Databases dictionary with 1 object(s):
	wind-example

In [16]:
generated_electricity_over_lifetime = 1000 # Amount of electricity generated by a wind turbine over its lifetime in kWh
share_of_wind_in_electricity_mix = 0.5

bd.Database('wind-example').write({
    ('wind-example', "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "unit": "kilogram",
    },
    ('wind-example', "coal"): {
        "type": "emission",
        "name": "coal",
        "unit": "kilogram",
    },
    ('wind-example', 'electricity-mix'): {
        'name': 'Electricity mix',
        'unit': 'kilowatt hour',
        'exchanges': [
            {
                'input': ('wind-example', 'electricity-production-wind'),
                'amount': share_of_wind_in_electricity_mix,
                'type': 'technosphere',
            },
            {
                'input': ('wind-example', 'electricity-production-coal'),
                'amount': 1-share_of_wind_in_electricity_mix,
                'type': 'technosphere',
            },
        ]
    },
    ('wind-example', 'electricity-production-coal'): {
        'name': 'Electricity production, coal',
        'unit': 'kilowatt hour',
        'exchanges': [
            {
                'input': ('wind-example', 'coal'),
                'amount': 1,
                'type': 'biosphere',
            },
            {
                'input': ('wind-example', 'CO2'),
                'amount': 1,
                'type': 'biosphere',
            },
        ]
    },
    ('wind-example', 'electricity-production-wind'): {
        'name': 'Electricity production, wind',
        'unit': 'kilowatt hour',
        'exchanges': [
            {
                'input': ('wind-example', 'electricity-production-wind'),
                'amount': 1,
                'type': 'production',
            },
            {
                'input': ('wind-example', 'wind-turbine-construction'),
                'amount': 1/generated_electricity_over_lifetime,
                'type': 'technosphere',
            },
            # maybe add maintenance, oil changes, ...
        ]
    },
    ('wind-example', 'wind-turbine-construction'): {
        'name': 'Wind turbine construction',
        'unit': 'unit',
        'exchanges': [
            {
                'input': ('wind-example', 'wind-turbine-construction'),
                'amount': 1,
                'type': 'production',
                'temporal_distribution': easy_timedelta_distribution(
                    start=-20,
                    end=0,
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=10,
                ),
            },
            {
                'input': ('wind-example', 'electricity-mix'),
                'amount': 1,
                'type': 'technosphere',
            },
            {
                'input': ('wind-example', 'eol-wind'),
                'amount': 1,
                'type': 'technosphere',
                'temporal_distribution': easy_timedelta_distribution(
                    start=0,
                    end=20,
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=10,
                ),
            },
            # aggregate the rest to direct co2 emissions
            {
                'input': ('wind-example', 'CO2'),
                'amount': 1,
                'type': 'biosphere',
            }
        ]
    },
    ('wind-example', 'eol-wind'): {
        'name': "End-of-life, wind turbine",
        'unit': 'unit',
        'exchanges': [
            {
                'input': ('wind-example', 'eol-wind'),
                'amount': 1,
                'type': 'production',
            },
            # aggregate the rest to direct co2 emissions
            {
                'input': ('wind-example', 'CO2'),
                'amount': 1,
                'type': 'biosphere',
            }
        ]
    },
})

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 7/7 [00:00<?, ?it/s]

Vacuuming database 


In [18]:
bd.Method(("GWP", "wind-example")).write([
    (('wind-example', "CO2"), 1),
])

In [19]:
lca = bc.LCA({('wind-example', 'electricity-mix'): 1}, ("GWP", "wind-example"))
lca.lci()
lca.lcia()
lca.score

0.5012506253720875